In [1]:
import os
import cv2
import numpy as np
import imutils
import random
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

C:\Users\Mahmoud Kholief\AppData\Roaming\Python\Python37\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [43]:
train_loc ='dataset/lallo/train'
test_loc = 'dataset/lallo/test'

In [44]:
# resize images 

trdata = ImageDataGenerator(rescale= 1./255,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            rotation_range=15,
                            fill_mode="nearest")
tsdata = ImageDataGenerator(rescale= 1./255)

traindata = trdata.flow_from_directory(directory = train_loc, target_size = (224,224), shuffle=True, batch_size = 8)
testdata = tsdata.flow_from_directory(directory = test_loc, target_size = (224,224), shuffle=False, batch_size = 1)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [45]:
traindata.class_indices

{'COVID': 0, 'Normal': 1}

In [46]:
baseModel = InceptionV3(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

In [47]:
# opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
H = model.fit(
	traindata,
	steps_per_epoch = traindata.samples // 8,
	validation_data = testdata,
	validation_steps = testdata.samples // 1,
	epochs=10)

Epoch 1/10
200/200 [==============================] - 47s 233ms/step - loss: 0.2679 - accuracy: 0.9112 - val_loss: 0.2343 - val_accuracy: 0.8875
Epoch 2/10
200/200 [==============================] - 34s 168ms/step - loss: 0.1596 - accuracy: 0.9413 - val_loss: 0.1259 - val_accuracy: 0.9500
Epoch 3/10
200/200 [==============================] - 34s 171ms/step - loss: 0.1346 - accuracy: 0.9506 - val_loss: 0.1271 - val_accuracy: 0.9550
Epoch 4/10
200/200 [==============================] - 34s 171ms/step - loss: 0.1528 - accuracy: 0.9531 - val_loss: 0.1342 - val_accuracy: 0.9525
Epoch 5/10
200/200 [==============================] - 37s 183ms/step - loss: 0.1402 - accuracy: 0.9550 - val_loss: 0.1069 - val_accuracy: 0.9575
Epoch 6/10
200/200 [==============================] - 35s 173ms/step - loss: 0.1332 - accuracy: 0.9450 - val_loss: 0.1298 - val_accuracy: 0.9575
Epoch 7/10
200/200 [==============================] - 34s 172ms/step - loss: 0.1253 - accuracy: 0.9550 - val_loss: 0.1110 - val_ac

In [49]:
model.evaluate(testdata, batch_size=8)

400/400 [==============================] - 9s 22ms/step - loss: 0.1338 - accuracy: 0.9500


[0.13375021517276764, 0.949999988079071]

In [50]:
predictions = model.predict(testdata, batch_size=8)
predictions = np.argmax(predictions, axis=1)

In [51]:
predictions

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,

In [52]:
testdata.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [53]:
predictions = model.predict(testdata, batch_size=8)
predictions = np.argmax(predictions, axis=1)

print(classification_report(testdata.classes, predictions,target_names=testdata.class_indices.keys()))

              precision    recall  f1-score   support

       COVID       0.95      0.95      0.95       200
      Normal       0.95      0.94      0.95       200

    accuracy                           0.95       400
   macro avg       0.95      0.95      0.95       400
weighted avg       0.95      0.95      0.95       400

